In [2]:
import pandas as pd
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [48]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

--2022-02-23 01:25:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-02-23 01:25:24 (10.3 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep=r"\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [6]:
#Number of rows 
df.count()

2643619

In [7]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'string'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'string'),
 ('total_votes', 'string'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [8]:

# Cast columns customer_id, product_id, star_rating, helpful_votes, total_votes to integer type
df = df.withColumn("customer_id", 
                                  df["customer_id"]
                                  .cast('int'))

df = df.withColumn("product_parent", 
                                  df["product_parent"]
                                  .cast('int'))

df = df.withColumn("star_rating", 
                                  df["star_rating"]
                                  .cast('int'))

df = df.withColumn("helpful_votes", 
                                  df["helpful_votes"]
                                  .cast('int'))

df = df.withColumn("total_votes", 
                                  df["total_votes"]
                                  .cast('int'))

df = df.withColumn("review_date", 
                                  df["review_date"]
                                  .cast('date'))
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
review_id_df = df.select(["review_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+----------+--------------+-----------+
|     review_id|product_id|product_parent|review_date|
+--------------+----------+--------------+-----------+
| REAKC26P07MDN|B00Q0K9604|     510387886| 2015-08-31|
|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672| 2015-08-31|
|R14QJW3XF8QO1P|B0084OHUIO|     902215727| 2015-08-31|
|R2HB7AX0394ZGY|B001GS71K2|     568880110| 2015-08-31|
| RGKMPDQGSAHR3|B004ABH1LG|     692846826| 2015-08-31|
|R1DJCVPQGCV66E|B00AX0LFM4|     590674141| 2015-08-31|
|R3V52EAWLPBFQG|B00DQFZGZ0|     688538603| 2015-08-31|
|R3DKO8J1J28QBI|B00DIRF9US|     742358789| 2015-08-31|
| R764DBXGRNECG|B00JRCBFUG|     869798483| 2015-08-31|
| RW1853GAT0Z9F|B000L3XYZ4|     501118658| 2015-08-31|
|R33GITXNUF1AD4|B00BOEXWFG|     454737777| 2015-08-31|
|R1H7AVM81TAYRV|B001HBBQKY|     420905252| 2015-08-31|
|R2ZOYAQZNNZZWV|B007O1FHB0|     302588963| 2015-08-31|
|R2FN1H3CGW6J8H|B001P3NU30|     525778264| 2015-08-31|
| RJB41Q575XNG4|B00ZP6HS6S|     414117299| 2015-08-31|
|R28W8BM15

In [14]:
products_df = df.select(["product_id", "product_title"]).distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00134HSYS|Special Edition P...|
|B00BS6NPBG|High Tech Pet 6-P...|
|B000F930FS|PetSafe ScoopFree...|
|B000FJ9QTW|Insight ActiviToy...|
|B000ALY0OQ|SmartCat Bootsie'...|
|B00Q8ETIZ0|Dogloveit Rubber ...|
|B002VU2BA4|Kragonfly Interch...|
|B00QA3K3QM|Attmu Retractable...|
|B005DGHUC2|Zoo Med Laborator...|
|B000XY7C7C|All Four Paws, Th...|
|B004UUE26O|Animal Planet PET...|
|B008APML2C|Chuckit Medium Ul...|
|B00ZJN7T8E|1 Half Portion Ja...|
|B00NRZC8LY|Hide-A-Toy Hallow...|
|B001P3NU4E|Virbac C.E.T. Enz...|
|B003TEQ2U6|Jolly Pets Jolly ...|
|B00DJSNF0M|Multipet Lenny th...|
|B00FXVFEQG|Bags on Board Dur...|
|B005ORDWYA|Just One Bite II ...|
|B003E77OG4|  Petmate Sky Kennel|
+----------+--------------------+
only showing top 20 rows



In [15]:
vine_df = df.select(["review_id", "star_rating","helpful_votes","total_votes"])
vine_df.show()

+--------------+-----------+-------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|
+--------------+-----------+-------------+-----------+
| REAKC26P07MDN|          5|            0|          0|
|R3NU7OMZ4HQIEG|          2|            0|          1|
|R14QJW3XF8QO1P|          5|            0|          0|
|R2HB7AX0394ZGY|          5|            0|          0|
| RGKMPDQGSAHR3|          5|            0|          0|
|R1DJCVPQGCV66E|          5|            0|          0|
|R3V52EAWLPBFQG|          3|            0|          0|
|R3DKO8J1J28QBI|          2|            0|          0|
| R764DBXGRNECG|          5|            1|          1|
| RW1853GAT0Z9F|          5|            0|          0|
|R33GITXNUF1AD4|          2|            0|          0|
|R1H7AVM81TAYRV|          1|            2|          2|
|R2ZOYAQZNNZZWV|          5|            0|          0|
|R2FN1H3CGW6J8H|          1|            0|          0|
| RJB41Q575XNG4|          5|            0|          3|
|R28W8BM15

In [39]:
customers_df = df.groupBy("customer_id").count()
customers_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   10270641|    1|
|   18365872|    1|
|   16711087|    1|
|   10742726|    2|
|   41169638|    1|
|   43622307|    1|
|   24540309|    2|
|   28258386|    1|
|   35329257|    2|
|   14552054|    1|
|   14529507|    5|
|   45392827|    5|
|   47282953|    1|
|    8201930|    1|
|   20109760|    2|
|   16405801|    4|
|   15056685|   21|
|   20840575|    2|
|   39048303|    1|
|    5596610|    1|
+-----------+-----+
only showing top 20 rows



In [41]:
customers_df = customers_df.withColumnRenamed("count","customer_count")

customers_df.printSchema()


root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [44]:
customers_df = customers_df.withColumn("customer_count", 
                                  customers_df["customer_count"]
                                  .cast('int'))

customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10270641|             1|
|   18365872|             1|
|   16711087|             1|
|   10742726|             2|
|   41169638|             1|
|   43622307|             1|
|   24540309|             2|
|   28258386|             1|
|   35329257|             2|
|   14552054|             1|
|   14529507|             5|
|   45392827|             5|
|   47282953|             1|
|    8201930|             1|
|   20109760|             2|
|   16405801|             4|
|   15056685|            21|
|   20840575|             2|
|   39048303|             1|
|    5596610|             1|
+-----------+--------------+
only showing top 20 rows



In [67]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.c0qbpuena9xq.us-east-1.rds.amazonaws.com:5432/mypostgresdb"
config = {"user":"root", 
          "password": "Leila0521!",
          "driver":"org.postgresql.Driver"}

In [68]:
# Write DataFrame to review_id table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [69]:
# Write DataFrame to review_id table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [70]:
# Write DataFrame to review_id table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [71]:
# Write DataFrame to review_id table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored